In [5]:
import pickle
import pandas as pd
import numpy as np
import math

import pubchempy as pcp
from rdkit.Chem import AllChem
from rdkit import Chem

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error,mean_absolute_error,median_absolute_error

from sklearn.metrics import make_scorer

import openpyxl

In [6]:
#pubchemからsmilesを取ってきてECFPのファイルを作成する
def make_compound_dic(df):
    id_list = df['Compound Identifier'].unique().tolist()
    properties = ['IUPACName', 'MolecularFormula', 'MolecularWeight', 'XLogP', 'TPSA', 'CanonicalSMILES']
    #get informations from pubChem
    chem_infos = []
    for cid in id_list:
        chem_info = pcp.get_properties(properties,cid)
        chem_infos.append(chem_info)

    #ECFP4
    radius = 2
    nBits = 4096
    morgan_fp = []
    for info_list in chem_infos:
        info = info_list[0]
        mol = Chem.MolFromSmiles(info['CanonicalSMILES'])
        fp = [i for i in AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits)]
        morgan_fp.append(fp)
    morgan_fp = np.array(morgan_fp)

    compound_dic = {}
    i = 0
    for info_list in chem_infos:
        info = info_list[0]
        compound_dic[info['CID']] = morgan_fp[i]
        i += 1
    
    with open('data/compound_dic.pickle',mode='wb') as f:
        pickle.dump(compound_dic,f)
        

def rmse_score(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = math.sqrt(mse)
    return rmse

def correlation_coeffcient(y_true, y_pred):
    correlation = np.corrcoef(list(y_true), list(y_pred))
    return correlation[0,1]

def str2int(s):
    if s == 'NaN':
        return 0
    else:
        return int(s)

def value2onehot(v):
    if v == 0:
        return 0
    else:
        return 1

def make_data(df,person,label,compound_dic):
    
    #使わないcolumnを削除
    df = df.drop('Odor', axis=1)
    df = df.drop('Replicate', axis=1)
    df = df.drop('Dilution', axis=1)

    #被験者を絞り込む処理
    sdf = df[df["subject #"]==str(person)].reset_index()
    sdf = sdf.drop('subject #', axis=1)
    sdf = sdf.drop('index', axis=1)
    sdf['Intensity'] = sdf['Intensity'].map({'low ':0,'high ':1})

    #int型に変換
    sdf['INTENSITY/STRENGTH'] = sdf['INTENSITY/STRENGTH'].apply(str2int)
    sdf['VALENCE/PLEASANTNESS'] = sdf['VALENCE/PLEASANTNESS'].apply(str2int)

    #説明変数とラベルデータを分離させる
    X = sdf.iloc[:,0:2]
    #print(X)
    Ys = sdf.iloc[:,2:]
    #print(Ys)

    #ECFP辞書検索
    ecfps = []
    for index in range(len(sdf)):
        data = sdf.loc[index]
        cid = int(data["Compound Identifier"])
        ecfps.append(list(compound_dic[cid]))
    
    #featureごとに列追加
    numOfColumn = len(ecfps[0])
    for c in range(numOfColumn):
        new = []
        for i in range(len(ecfps)):
            new.append(ecfps[i][c])
        X['f'+str(c)] = new
    
    #不要columnを削除
    X = X.drop('Compound Identifier', axis=1)

    #ラベルのset
    Y = Ys[label]
    Y = Y.apply(str2int)
    
    return X,Y,Ys


#モデルの評価関数 
def evaluate(clf,X,Y,k,label,make_df = 1):
    score_funcs = {
        'correlation':make_scorer(correlation_coeffcient),
        'rmse': make_scorer(rmse_score),
        'MedAE':'neg_median_absolute_error',
        'R2':'r2',
        'MAE':'neg_mean_absolute_error'
    }
    
    kf=KFold(n_splits=k, shuffle=True, random_state=0)
    score=cross_validate(clf, X, Y, cv = kf,
                        scoring=score_funcs)
                        #,return_train_score=True)
    df = pd.DataFrame(score)
    df.loc[label+"_mean"]=df.mean()
    return df

In [7]:
#trainDataの読み込み
train_set = []
with open('data/TrainSet.txt') as file:
    for f in file:
        line = f.split('\t')
        line[-1] = line[-1].split('\n')[0]
        train_set.append(line)
        
df = pd.DataFrame(train_set[1:],columns = train_set[0])

#make_compound_dic(df)

#化合物データの読み込み
with open('data/compound_dic.pickle','rb') as f:
    compound_dic = pickle.load(f)

In [8]:
X,Y,ori= make_data(df,1,'GRASS',compound_dic)

In [9]:
X

,Intensity,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f4086,f4087,f4088,f4089,f4090,f4091,f4092,f4093,f4094,f4095
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
712,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
713,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
714,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
labels = list(df.iloc[:,6:].columns)
labels[18]

'GRASS'

In [14]:
oridf = df
for person in range(49,50):#人それぞれに対して
    rdf = pd.DataFrame()
    for label in labels:#それぞれのラベルに対して回帰
        df = oridf
        X,Y,ori= make_data(df,person,label,compound_dic)
        clf = RandomForestClassifier(n_estimators=100,max_features='auto',oob_score=False,n_jobs=1,random_state=0)
        evaluate_df = evaluate(clf,X,Y,5,label)
        rdf = pd.concat([rdf, evaluate_df.loc[label+"_mean"]], axis=1)
    rdf.to_excel('result/'+str(person)+'_predict_result.xlsx', sheet_name=str(person)+'_predict_result')

/Users/Haruki/anaconda3/envs/my-rdkit-env/lib/python3.6/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/Haruki/anaconda3/envs/my-rdkit-env/lib/python3.6/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/Haruki/anaconda3/envs/my-rdkit-env/lib/python3.6/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/Haruki/anaconda3/envs/my-rdkit-env/lib/python3.6/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/Haruki/anaconda3/envs/my-rdkit-env/lib/python3.6/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/Haruki/anaconda3/envs/my-rdkit-env/lib/python3.6/site-packages/numpy/lib/funct

In [15]:
pd.read_excel('result/49_predict_result.xlsx')

,Unnamed: 0,INTENSITY/STRENGTH_mean,VALENCE/PLEASANTNESS_mean,BAKERY_mean,SWEET_mean,FRUIT_mean,FISH_mean,GARLIC_mean,SPICES_mean,COLD_mean,...,ACID_mean,WARM_mean,MUSKY_mean,SWEATY_mean,AMMONIA/URINOUS_mean,DECAYED_mean,WOOD_mean,GRASS_mean,FLOWER_mean,CHEMICAL_mean
0,fit_time,2.184442,2.397632,0.343526,1.388876,0.691941,0.402037,0.462539,1.330094,0.796539,...,0.875236,0.964705,1.571841,1.259505,1.487125,0.733920,0.676296,0.454282,0.931112,1.703408
1,score_time,0.028091,0.028822,0.019189,0.024619,0.021735,0.018950,0.019216,0.023399,0.021499,...,0.020565,0.022507,0.024103,0.025004,0.023785,0.022972,0.020378,0.018348,0.022361,0.028489
2,test_correlation,0.248050,0.215762,0.613278,0.354382,0.213480,0.033649,0.008227,0.253269,-0.015252,...,0.017273,-0.022441,0.175889,0.004815,0.052443,0.431523,0.083538,-0.012950,0.128344,0.196023
3,test_rmse,37.851631,29.709177,1.421572,11.885354,11.772383,1.897660,6.333005,7.478224,6.116019,...,1.745698,4.104936,16.736764,11.404215,10.542181,5.060194,4.107258,7.825433,8.294097,21.184899
4,test_MedAE,-23.600000,-17.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,test_R2,-0.578729,-0.680080,0.069674,-0.383896,-0.524221,-2.146859,-0.594382,-0.665011,-0.503235,...,-0.675525,-2.240227,-1.921444,-0.946290,-0.608295,-4.252477,-0.861842,-11.192743,-1.599873,-0.318746
6,test_MAE,-28.965414,-22.780750,-0.212111,-3.856828,-2.248883,-0.253914,-0.878564,-2.269114,-1.370765,...,-0.525029,-1.073495,-6.264870,-3.567249,-3.118007,-0.930458,-0.778011,-0.978972,-1.890142,-8.867444


# データセットに関して

- "Compound Identifier"には、PubChemのCompound Identification（ユニークな化学構造の識別子）が含まれている。

- 2 番目の列（"Odor"）には、化学物質の同義語が含まれています。→いらない

- 3番目の列（「複製」）は、その刺激が2回テストされた20の刺激の一部であるかどうかを示します。→ いらないことにした

- 第４の列（「強度」）は、そのデータが高強度（その臭気の希釈度が低い）での臭気のデータであるか、低強度（その臭気の希釈度が高い）での臭気のデータであるかを示す。

- 第5列（「希釈」）には、臭気の希釈度が記載されています。各臭気は2つの異なる希釈で表示されます。弱い臭気は、すべての臭気の低い希釈がほぼ等しい強度を持つように、強い臭気よりも少なく希釈されています。すべての臭気の高希釈もまた、ほぼ等しい強度を持っています。(等強度とは、非常に大まかにしか言えません。化学物質の中には全く臭気のないものもありますので、明らかに他のものと等しくなるように希釈することはできません)。
→4列目だけで十分とした

- 第6列（「被験者#」）には、被験者の識別子、1から49までの番号が記載されている。
第７〜２７列には、４９人の被験者が匂いのテスト中に提供した知覚データが含まれている。
被験者は、与えられた匂いを評価する前に、匂いを嗅いだときに何かの匂いがするかどうかを尋ねられた。
被験者が何も臭わないと答えた場合、強度評価は自動的に「０」に設定され、他の評価は空白のままにされた。
時折、被験者はこの最初の質問に「はい」と答え、何かのにおいがすることを示したが、
その後、強度の評価を「0」とした。これらの被験者は、他のすべての評価も完了しました。
これが、強度の評価が "0 "の場合、ほとんどの場合、他のデータがないことがわかりますが、
いくつかのケースでは、強度が "0 "と評価されていても、完全なデータセットを見つけることができる。

- 第7列（「INTENSITY/STRENGTH」）には、被験者がどの程度の強さで匂いを知覚したかについての知覚データが含まれています。被験者は、０から１００までの尺度を使用し、１００は「非常に強い」、０は「非常に弱い」である。
- 8番目の列（VALENCE/PLEASANTNESS）は、被験者が嗅いだ匂いがどの程度気持ち良いか不快かについての知覚データを含んでいる。被験者は、０〜１００の尺度を使用し、１００は「非常に快」であり、０は「非常に不快」である。

- 列9-27は、被験者がどのように臭気を感じたかについての知覚を19個の知覚記述子の標準リストと一致させたデータを含む。
'BAKERY',
 'SWEET',
 'FRUIT',
 'FISH',
 'GARLIC',
 'SPICES',
 'COLD',
 'SOUR',
 'BURNT',
 'ACID',
 'WARM',
 'MUSKY',
 'SWEATY',
 'AMMONIA/URINOUS',
 'DECAYED',
 'WOOD',
 'GRASS',
 'FLOWER',
 'CHEMICAL'
被験者は0-100の尺度を使用し、100は「非常に」、0は「全く」とした。
注意: 被験者が何も匂わなかった場合，強度はヌル値を持ち，他の知覚記述子は値を持たないままにした．

In [7]:
#データセット 
df.head()

,Compound Identifier,Odor,Replicate,Intensity,Dilution,subject #,INTENSITY/STRENGTH,VALENCE/PLEASANTNESS,BAKERY,SWEET,...,ACID,WARM,MUSKY,SWEATY,AMMONIA/URINOUS,DECAYED,WOOD,GRASS,FLOWER,CHEMICAL
0,126,4-Hydroxybenzaldehyde,,low,"""1/1,000 """,1,7,62,0,0,...,0,0,0,21,0,0,0,0,0,0
1,126,4-Hydroxybenzaldehyde,,high,"""1/10""",1,37,60,0,72,...,0,0,0,0,0,0,0,0,0,0
2,126,4-Hydroxybenzaldehyde,,low,"""1/1,000 """,2,55,89,0,33,...,0,0,0,0,0,0,0,0,0,5
3,126,4-Hydroxybenzaldehyde,,high,"""1/10""",2,64,71,0,9,...,0,0,0,0,0,0,0,0,0,7
4,126,4-Hydroxybenzaldehyde,,low,"""1/1,000 """,3,89,68,0,62,...,0,62,0,0,0,0,0,0,0,0


In [8]:
#被験者1に関する説明変数
X

,Intensity,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f4086,f4087,f4088,f4089,f4090,f4091,f4092,f4093,f4094,f4095
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
712,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
713,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
714,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# 1人に対して予測(シングル)

- 被験者number 1 に対してモデルの作成

- モデルはランダムフォレスト(もう一つとしてLasso回帰をやる)
- 5-foldでcoross_validate()でモデルの作成、評価
- 二乗平均平方根誤差 , 平均絶対誤差　, Medium Absolute error,決定係数の指標で結果を見る

In [9]:
#crossvalidate()
evaluate_df

,fit_time,score_time,test_correlation,test_rmse,test_MedAE,test_R2,test_MAE
0,1.029539,0.019634,0.162255,23.039519,-0.0,-0.772540,-8.166667
1,0.906194,0.018802,0.182191,25.673855,-0.0,-0.225285,-9.510490
2,0.942212,0.019771,0.249611,23.681009,-0.0,-0.278241,-8.482517
3,0.959785,0.019288,0.150248,19.124546,-0.0,-0.760665,-6.993007
4,0.941919,0.019343,0.193806,22.760535,-0.0,-0.186959,-8.643357
CHEMICAL_mean,0.955930,0.019368,0.187622,22.855893,0.0,-0.444738,-8.359207


In [10]:
pd.read_excel('result/1_predict_result.xlsx')

,Unnamed: 0,INTENSITY/STRENGTH_mean,VALENCE/PLEASANTNESS_mean,BAKERY_mean,SWEET_mean,FRUIT_mean,FISH_mean,GARLIC_mean,SPICES_mean,COLD_mean,...,ACID_mean,WARM_mean,MUSKY_mean,SWEATY_mean,AMMONIA/URINOUS_mean,DECAYED_mean,WOOD_mean,GRASS_mean,FLOWER_mean,CHEMICAL_mean
0,fit_time,2.050145,2.246390,0.457057,2.020746,0.854743,0.314640,0.331097,1.156751,0.218853,...,0.814777,0.176223,1.168773,0.669660,0.343889,0.821761,0.357770,0.242341,0.359119,1.208995
1,score_time,0.028590,0.025837,0.026191,0.031952,0.022521,0.023788,0.015375,0.024090,0.016692,...,0.020896,0.015463,0.024268,0.021597,0.018634,0.022698,0.019537,0.016766,0.018420,0.024697
2,test_correlation,0.148315,0.242148,0.185699,0.098524,0.195405,-0.010499,0.652247,0.010530,NaN,...,0.239024,NaN,0.112330,0.086248,-0.010690,0.105541,-0.007978,NaN,-0.013559,0.285491
3,test_rmse,46.305788,28.514786,12.444180,36.750252,21.819744,10.020421,6.856713,26.807370,1.347946,...,19.092166,3.647468,34.124218,23.484279,9.858967,23.108040,6.756720,1.953854,8.365004,28.261574
4,test_MedAE,-30.400000,-17.700000,0.000000,-3.600000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,test_R2,-0.814755,-0.513881,-0.647327,-0.679664,-0.412701,-0.869635,0.188309,-0.873111,0.596396,...,-0.428968,0.397193,-0.506900,-0.440902,-0.071412,-0.512698,-0.011548,0.396798,-7.862610,-0.247483
6,test_MAE,-35.456915,-22.000408,-2.400466,-22.081362,-7.025602,-1.363258,-0.959130,-11.575311,-0.163636,...,-6.088899,-0.326301,-14.373329,-6.549932,-1.412646,-6.688753,-0.733460,-0.180264,-1.417560,-12.358285


In [11]:
pd.read_excel('result/1_predict_result.xlsx').loc[2]

Unnamed: 0                   test_correlation
INTENSITY/STRENGTH_mean              0.148315
VALENCE/PLEASANTNESS_mean            0.242148
BAKERY_mean                          0.185699
SWEET_mean                          0.0985239
FRUIT_mean                           0.195405
FISH_mean                          -0.0104987
GARLIC_mean                          0.652247
SPICES_mean                         0.0105299
COLD_mean                                 NaN
SOUR_mean                          -0.0252457
BURNT_mean                          -0.012939
ACID_mean                            0.239024
WARM_mean                                 NaN
MUSKY_mean                            0.11233
SWEATY_mean                         0.0862484
AMMONIA/URINOUS_mean               -0.0106896
DECAYED_mean                         0.105541
WOOD_mean                         -0.00797789
GRASS_mean                                NaN
FLOWER_mean                        -0.0135588
CHEMICAL_mean                     